<a href="https://colab.research.google.com/github/astrid12345/recyclo/blob/mk_test_train/src/train_YOLO11n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a bare-bones file that demonstrates downloading a dataset from google drive and using it to train a YOLOv8 model.

Consider this a placeholder only - if you're training a model for real, please overhaul this file however you deem necessary!

In [ ]:
!pip install -U ultralytics gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
google_drive_file_id = '1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg'  # <-- From sharing link in google drive, eg https://drive.google.com/file/d/1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg/view?usp=sharing
google_drive_file_name = '20250623_merged_yolo_reshuffled.zip'

In [ ]:
import zipfile
from pathlib import Path
import gdown
from ultralytics import YOLO
import yaml

project_root = Path('/content')
dataset_root_name = 'yolo_dataset'
output_path = project_root / google_drive_file_name
extract_dir = project_root / dataset_root_name
dataset_root_path = Path('/content') / dataset_root_name
yaml_path = dataset_root_path / 'data.yaml'

# Download from Google Drive and extract
gdown.download(id=google_drive_file_id, output=str(output_path), quiet=False)
print(f"Dataset downloaded from google drive: {output_path}\n")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Downloading...
From (original): https://drive.google.com/uc?id=1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg
From (redirected): https://drive.google.com/uc?id=1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg&confirm=t&uuid=ec2cb28c-446a-429e-aa34-14adf707cbe8
To: /content/20250623_merged_yolo_reshuffled.zip
100%|██████████| 3.98G/3.98G [01:06<00:00, 59.4MB/s]

Dataset downloaded from google drive: /content/20250623_merged_yolo_reshuffled.zip



In [ ]:
print("Extracting...\n")
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Dataset extracted to: {extract_dir}\n", )


Extracting...

Dataset extracted to: /content/yolo_dataset



In [ ]:

# Update path in data.yaml to work with where we've unzipped it
print("Updating data.yaml path...\n")
with yaml_path.open('r') as file:
    data = yaml.safe_load(file)

data['path'] = str(dataset_root_path)
data['train'] = str(dataset_root_path / 'images' / 'train')
data['val'] = str(dataset_root_path / 'images' / 'val')
data['test'] = str(dataset_root_path / 'images' / 'test')

with yaml_path.open('w') as file:
    yaml.dump(data, file)

print(f"Updated path in data.yaml to: {data['path']}")

Updating data.yaml path...

Updated path in data.yaml to: /content/yolo_dataset


In [ ]:
# Sanity check that it's possible to train a model with the dataset
dataset_yaml = Path('/content/yolo_dataset/data.yaml')
model = YOLO('yolo11n.pt')
results = model.train(data=str(dataset_yaml), epochs=100, imgsz=640)

print(f"Training complete. Results saved to: {model.trainer.save_dir}")

# Get the directory where results are saved
save_dir = Path(model.trainer.save_dir)
print(f"Training complete. Results saved to: {save_dir}")

# Zip the results directory
zip_path = save_dir.with_suffix('.zip')
shutil.make_archive(str(save_dir), 'zip', root_dir=save_dir)

# Download the zipped results
files.download(str(zip_path))

100%|██████████| 5.35M/5.35M [00:00<00:00, 298MB/s]


Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, crop_fraction=1.0, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=600, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, label_smoothing=0.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov8n-c3k2-6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

100%|██████████| 755k/755k [00:00<00:00, 82.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

 23        [16, 19, 22]  1    430867  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2780.1±1445.5 MB/s, size: 424.1 KB)


...

val: Scanning /content/yolo_dataset/labels/val... 434 images, 0 backgrounds, 0 corrupt: 100%|██████████| 434/434 [00:07<00:00, 58.54it/s]

val: New cache created: /content/yolo_dataset/labels/val.cache



val: Caching images (4.9GB Disk): 100%|██████████| 434/434 [00:54<00:00,  8.03it/s]


WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
WARNING ⚠️ 'crop_fraction' is deprecated and will be removed in in the future.
Plotting labels to exp10-new/yolov8n-c3k2-6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.001), 87 bias(decay=0.0)


AssertionError: yolo11n.pt training to 600 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolo11n.pt'